In [1]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchdata


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


you should either use  pandas==1.2.5 or datase=2.11

In [5]:
pip install pandas==1.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas: filename=pandas-1.2.5-cp310-cp310-linux_x86_64.whl size=11024661 sha256=5db1cb2aaede79dc2133f8879c1fb9f00452eac1337147514882b9167ce33d34
  Stored in directory: /root/.cache/pip/wheels/80/c4/45/fbb3bb8c610988624e1a0cb4c55ac7409fba8c9b3fbaadcd9e
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
esda 2.5.0 requires pandas>1

In [6]:
import torch
import torchdata
import transformers
import datasets
import pandas as pd

In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

In [8]:
huggingface_dataset_name= "knkarthick/dialogsum"
dataset= load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/knkarthick--dialogsum-1aed23a5f481e688/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# **Summarize Dialogue without Prompt Engineering**

Print a couple of  dialogues with thier baseline summaries

In [10]:
example_indices=[20,100]
dash_line  = '-'.join('' for x in range (100))
for i, index in enumerate(example_indices) : 
    print(dash_line)
    print('example', i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.
----------------------------------------------------

To perform encoding/decoding, you need to work with text in a tokenized form.

**Tokenization:**
is the process of splitting the text into smaller units that can be processed by the LLM (in this case FLAN-T5)
Load the  FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with .from_pretrained() method

In [11]:

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

**Test the tokenizer encoding and decoding s simple sentence**

In [12]:
sentence ="what are you doing?"
sentence_encoded = tokenizer(sentence, return_tensors='pt')
sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0],
                                    skip_special_tokens=True)

print("Encoded sentence:")
print(sentence_encoded["input_ids"][0])
print("\nDecoded sentence")
print(sentence_decoded)

Encoded sentence:
tensor([125,  33,  25, 692,  58,   1])

Decoded sentence
what are you doing?


Dialogue summarization using FLAN-T5 without prompt engineering

In [13]:
for i, index in enumerate(example_indices): 
    dialogue= dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    inputs = tokenizer(dialogue, return_tensors='pt')
    output =tokenizer.decode(model.generate(inputs["input_ids"],
                                          max_new_tokens = 50,)
                            [0], 
                             skip_special_tokens= True)
    print(dash_line)
    print('example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.
------------------------------------------------------

# **Prompt engineering :**
 is an act of a human changing the prompt(input) to improve the response for a given task.

**Zero Shot Inference with an Instruction Prompt :**

In order to instruct the model to perform a task (dialogue summarization) you can take the dialogue and convert it into an instruction prompt. This is often called Zero Shot Inference.

In [14]:
for i, index in enumerate(example_indices): 
    dialogue= dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt =f"""
    
Summarize the following conversation.

{dialogue}

summary: 
    """
    inputs= tokenizer(prompt, return_tensors='pt')
   
    output =tokenizer.decode(model.generate(inputs["input_ids"],
                                          max_new_tokens = 50,)
                            [0], 
                             skip_special_tokens= True)
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - Zero shot:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    
Summarize the following conversation.

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.

summary:

In [15]:
for i, inedex in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    
    prompt =f"""
Dialogue
{dialogue}
what wasgoing on ?
    """
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
    model.generate(
    inputs["input_ids"],
    max_new_tokens=50,)[0],
    skip_special_tokens = True)
    
    print(dash_line)
    print('example', i+1)
    print(dash_line)
    print(f'Imput promt:\n{prompt}')
    print(dash_line)
    print(f'Baseline Human Summary:\n{summary}\n')
    print(dash_line)
    print(f'Model Generation Zero shot:\n{output}\n')


---------------------------------------------------------------------------------------------------
example 1
---------------------------------------------------------------------------------------------------
Imput promt:

Dialogue
#Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person2#: What was the problem that time?
#Person1#: The feeling was all wrong, Mike. She is telling you that she doesn't want to see you any more, but I want to get more anger from you. You're acting hurt and sad, but that's not how your character would act in this situation.
#Person2#: But Jason and Laura have been together for three years. Don't you think his reaction would be one of both anger and sadness?
#Person1#: At this point, no. I think he would react the way most guys would, and then later on, we would see his real feelings.
#Person2#: I'm not so sure about that.
#Person1#: Let's try it my way, and you can see how you feel when you're saying your lines. After that, if it sti

# **Summarize Dialogue With One Shot & Few Shot Inference**

One Shot and Few Shot are the practice of providing an LLM with either one or more full examples of prompt-response pairs that match your task before your actual prompt that you want completed. This called in context learning and puts your model into a state that understands your specific task

In [16]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt=''
    for index in example_indices_full : 
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        prompt +=f"""
Dialogue :
{dialogue}
what was going on?
{summary}
        """
        dialogue =dataset['test'][example_index_to_summarize]['dialogue']
        prompt +=f"""
Dialogue : 
{dialogue}
what was going on?
        """
        return(prompt)

# **One Shot Inference**

In [22]:
example_indices_full= [10]
example_index_to_summarize = 512
one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


Dialogue :
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
what was going on?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.
        
Dialogue : 
#Person1#: Excuse me. I'm a stranger here and lost my way.
#Person2#: Can I help you?
#Person1#: Sure. Can I get to the central department stall this way?
#Person2#: uh. . . Yes. Turn right

In [23]:
summary = dataset['test'][example_index_to_summarize]['summary']
iputs = tokenizer(one_shot_prompt, return_tensors='pt')
ouput =tokenizer.decode(
model.generate(
inputs["input_ids"], 
max_new_tokens=50,)
[0], 
skip_special_tokens =True)
print(dash_line)
print(f'Baseline Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Generation -- one shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary:
#Person1# asks #Person2# the way to the central department stall and the national bank.

---------------------------------------------------------------------------------------------------
Model Generation -- one shot:
The feeling was all wrong, Mike.


# **Few Shot Prompt**

In [41]:
example_indices_full= [55,100,512]
example_index_to_summarize = 512
few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


Dialogue :
#Person1#: Is anybody in?
#Person2#: How can I help you?
#Person1#: I have a headache.
#Person2#: Let me take your temperature with a thermometer.
#Person1#: OK.
#Person2#: I think you have a small fever.
#Person1#: I thought so. I felt dizzy this morning.
#Person2#: You should've called in sick! Next time, have either of your parents call the school office.
what was going on?
#Person2# finds that #Person1# has a fever and says #Person1# should've called in sick earlier.
        
Dialogue : 
#Person1#: Excuse me. I'm a stranger here and lost my way.
#Person2#: Can I help you?
#Person1#: Sure. Can I get to the central department stall this way?
#Person2#: uh. . . Yes. Turn right, then take the second turning on your left.
#Person1#: Is it far?
#Person2#: It's about fifteen minutes'walk. That's all.
#Person1#: And do you know where the national bank is?
#Person2#: Yes. It's on this street. Keep walking for two blocks and it's on the corner on the right.
#Person1#: Thanks very

In [42]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs =tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
max_new_tokens = 50,)
[0],
skip_special_tokens =True)
print(dash_line)
print(f'Baseline Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Generation -- few shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary:
#Person1# asks #Person2# the way to the central department stall and the national bank.

---------------------------------------------------------------------------------------------------
Model Generation -- few shot:
#Person1# is a stranger and lost his way. #Person2# will help him.


# **Generative Configuration Paramaters for Inference**

You can change the configuration parameters of the generate()  methode to see a different output from the LLM.
Putting the parameter do_sample = True, you activate various decoding strategies which influance the next token from the probability distribution over the entier vocabulary. You can adjust the outputs changing temperature and ther parameters( top_k, top_p)

In [43]:
generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenrationConfig(max_new_tokens = 10)
#generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# is a stranger and lost his way. #Person2# will help him.
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.



In [45]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenrationConfig(max_new_tokens = 10)
generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# is a stranger and lost his way. #Person2# will help him.
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.



In [46]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenrationConfig(max_new_tokens = 10)
#generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature=0.1)
generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# needs help from Person2 to get to the central department stall.
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.



In [47]:
#generation_config = GenerationConfig(max_new_tokens=50)
generation_config = GenerationConfig(max_new_tokens = 10)
#generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# is a stranger and
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.



In [48]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens = 10)
generation_config = GenerationConfig(max_new_tokens = 10, do_sample = True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# is a stranger and
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.



In [49]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens = 10)
generation_config = GenerationConfig(max_new_tokens = 100, do_sample = True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample= True, temperature=0.5)
#generation_config= GenerationConfig(max_new_tokens = 50,do_sample =True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
output=tokenizer.decode(
model.generate(
inputs["input_ids"],
generation_config=generation_config,)
[0],
skip_special_tokens= True)
print(dash_line)
print(f'Model Generation few_shot:\n{output}')
print(dash_line)
print(f'Baseline Human summary : \n{summary}\n')

---------------------------------------------------------------------------------------------------
Model Generation few_shot:
#Person1# is a stranger and lost his way. #Person2# will help him.
---------------------------------------------------------------------------------------------------
Baseline Human summary : 
#Person1# asks #Person2# the way to the central department stall and the national bank.

